In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from io import StringIO
import datetime

In [2]:
%matplotlib inline
%reload_kedro
%load_ext nb_black

2022-03-29 17:20:58,228 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-03-29 17:20:58,279 - kedro.config.config - INFO - Config from path `/Users/juanazuero/Documents/Transporte/Transmilenio/transmi/conf/local` will override the following existing top-level config keys: dic_estacion_linea_int, dic_estacion_linea_raw, validaciones_troncal, validaciones_troncal_log_int, validaciones_troncal_log_raw
2022-03-29 17:20:58,283 - root - INFO - ** Kedro project transmi
2022-03-29 17:20:58,284 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2022-03-29 17:20:58,320 - blib2to3.pgen2.driver - INFO - Generating grammar tables from /opt/anaconda3/envs/transmilenio/lib/python3.8/site-packages/blib2to3/Grammar.txt
2022-03-29 17:20:58,334 - blib2to3.pgen2.driver - INFO - Writing grammar tables to /Users/juanazuero/Library/Caches/black/22.1.0/Grammar3.8.0.final.0.pickle
2022-03-29 17:20:58,335 - 

<IPython.core.display.Javascript object>

In [18]:
validaciones_troncal_log = catalog.load("validaciones_troncal_log_raw")
dic_estacion_linea = catalog.load("dic_estacion_linea_raw")

2022-03-29 15:38:43,597 - kedro.io.data_catalog - INFO - Loading data from `validaciones_troncal_log_raw` (JSONDataSet)...
2022-03-29 15:38:46,927 - kedro.io.data_catalog - INFO - Loading data from `dic_estacion_linea_raw` (JSONDataSet)...


<IPython.core.display.Javascript object>

In [46]:
start_validaciones_troncal = catalog.load("params:start_validaciones_troncal")
extraction_day = catalog.load("params:extraction_day")

2022-03-29 15:51:38,671 - kedro.io.data_catalog - INFO - Loading data from `params:start_validaciones_troncal` (MemoryDataSet)...
2022-03-29 15:51:38,673 - kedro.io.data_catalog - INFO - Loading data from `params:extraction_day` (MemoryDataSet)...


<IPython.core.display.Javascript object>

In [56]:
BASE_VALIDACIONES_REQUEST = "https://storage.googleapis.com/validaciones_tmsa/"

<IPython.core.display.Javascript object>

In [49]:
if extraction_day == "next":
    end_validaciones_troncal = datetime.datetime.today().date() - datetime.timedelta(
        days=2
    )
    validaciones_range = [
        datetime.datetime.date(x)
        for x in pd.date_range(start_validaciones_troncal, end_validaciones_troncal)
    ]
    first_failed = None
    for day in validaciones_range:
        if not day in validaciones_troncal_log.keys():
            extraction_day = day
            break
        elif "fail" == validaciones_troncal_log[str(day)][:4] and first_failed is None:
            first_failed = day
    if extraction_day == "next":
        if first_failed is not None:
            extraction_day = first_failed
        else:
            return pd.DataFrame(), validaciones_troncal_log, dic_estacion_linea
    

<IPython.core.display.Javascript object>

In [67]:
response = requests.get(
    f"{BASE_VALIDACIONES_REQUEST}ValidacionTroncal/validacionTroncal{str(extraction_day).replace('-', '')}.csv"
)
if response.status_code == 200:
    data_str_io = StringIO(response.text)
    response_pd = pd.read_csv(data_str_io, sep=",")
    dic_estacion_linea_response = (
        response_pd[["Estacion_Parada", "Linea"]]
        .drop_duplicates()
        .set_index("Estacion_Parada")
        .to_dict()["Linea"]
    )
    response_pd.drop(
        [
            "ID_Vehiculo",
            "Ruta",
            "Tipo_Vehiculo",
            "Operador",
            "Hora_Pico_SN",
            "Dispositivo",
            "Day_Group_Type",
            "Fecha_Clearing",
            "Tipo_Tarifa",
            "Sistema",
            "Saldo_Despues_Transaccion",
            "Fase",
            "Linea",
        ],
        axis=1,
        inplace=True,
    )
    response_pd["Fecha_Transaccion"] = pd.to_datetime(
        response_pd["Fecha_Transaccion"], format="%Y-%m-%d %H:%M:%S UTC", utc=True
    )
    response_pd["day"] = response_pd["Fecha_Transaccion"].apply(lambda x: x.date)
    try:
        assert list(response_pd.columns) == [
            "Acceso_Estacion",
            "Emisor",
            "Estacion_Parada",
            "Fecha_Transaccion",
            "Nombre_Perfil",
            "Numero_Tarjeta",
            "Saldo_Previo_a_Transaccion",
            "Tipo_Tarjeta",
            "Valor",
            "day",
        ], "Database inconsistency"
    except AssertionError:
        print("Database inconsistency")
        validaciones_troncal_log[str(extraction_day)] = "fail - Database inconsistency"
        # return pd.DataFrame(), validaciones_troncal_log, dic_estacion_linea
else:
    reason = response.reason
    print(f"Failed request: {reason}")
    validaciones_troncal_log[str(extraction_day)] = f"fail - {reason}"
    # return pd.DataFrame(), validaciones_troncal_log, dic_estacion_linea

sigue


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
date_prueba = pd.to_datetime("2021-01-01 04:06:07 UTC")

<IPython.core.display.Javascript object>

In [14]:
date_prueba

Timestamp('2021-01-01 04:06:07+0000', tz='UTC')

<IPython.core.display.Javascript object>

In [20]:
date_prueba.astimezone(tz="America/Bogota")

datetime.date(2020, 12, 31)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [92]:
# response_pd["Fecha_Clearing"] = pd.to_datetime(
#    response_pd["Fecha_Clearing"], format="%Y-%m-%d"
# )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [98]:
response_pd.drop("Emisor", axis=1, inplace=True)

<IPython.core.display.Javascript object>

Database inconsistency


<IPython.core.display.Javascript object>

In [104]:
response_pd.iloc[0]

Acceso_Estacion                                 (04) Acceso Peatonal Occidental
Estacion_Parada                                         (06000) Portal Eldorado
Fecha_Transaccion                                     2020-01-01 16:47:21+00:00
Nombre_Perfil                                                      (001) Adulto
Numero_Tarjeta                c54bf7ae7353d0b8038f4fe44ca5ae96ce3580287aa052...
Saldo_Previo_a_Transaccion                                                 7400
Tipo_Tarjeta                                                       tullave Plus
Valor                                                                      2400
day                                                                  2020-01-01
Name: 0, dtype: object

<IPython.core.display.Javascript object>

In [101]:
response_pd.iloc[0]

Acceso_Estacion                                 (04) Acceso Peatonal Occidental
Estacion_Parada                                         (06000) Portal Eldorado
Fecha_Transaccion                                     2020-01-01 16:47:21+00:00
Nombre_Perfil                                                      (001) Adulto
Numero_Tarjeta                c54bf7ae7353d0b8038f4fe44ca5ae96ce3580287aa052...
Saldo_Previo_a_Transaccion                                                 7400
Tipo_Tarjeta                                                       tullave Plus
Valor                                                                      2400
day                                                                  2020-01-01
Name: 0, dtype: object

<IPython.core.display.Javascript object>

In [8]:
prueba_dict = {1: 2, 3: 4, 5: 6}

<IPython.core.display.Javascript object>

In [10]:
catalog.save("dic_estacion_linea", prueba_dict)

2022-03-29 15:08:43,029 - kedro.io.data_catalog - INFO - Saving data to `dic_estacion_linea` (JSONDataSet)...


<IPython.core.display.Javascript object>

In [105]:
dic_estacion_linea.update(dic_estacion_linea_response)

<IPython.core.display.Javascript object>

In [106]:
dic_estacion_linea

{'(06000) Portal Eldorado': '(11) Zona K Calle 26',
 '(06001) Modelia': '(11) Zona K Calle 26',
 '(06002) Normandía': '(11) Zona K Calle 26',
 '(06100) Av. Rojas': '(11) Zona K Calle 26',
 '(06101) El Tiempo': '(11) Zona K Calle 26',
 '(06102) Salitre El Greco': '(11) Zona K Calle 26',
 '(06103) CAN': '(11) Zona K Calle 26',
 '(06104) Gobernación': '(11) Zona K Calle 26',
 '(06105) Quinta Paredes': '(11) Zona K Calle 26',
 '(06106) Recinto Ferial': '(11) Zona K Calle 26',
 '(06107) Ciudad Universitaria': '(11) Zona K Calle 26',
 '(06108) Concejo de Bogotá': '(11) Zona K Calle 26',
 '(06109) Centro Memoria': '(11) Zona K Calle 26',
 '(06111) Universidades': '(11) Zona K Calle 26',
 '(10000) Portal 20 de Julio': '(12) Zona L Carrera 10',
 '(10001) Country Sur': '(12) Zona L Carrera 10',
 '(10002) Av. Primero de Mayo': '(12) Zona L Carrera 10',
 '(10003) Ciudad Jardín': '(12) Zona L Carrera 10',
 '(10004) Policarpa': '(12) Zona L Carrera 10',
 '(10010) San Bernardo': '(12) Zona L Carrera 

<IPython.core.display.Javascript object>

In [3]:
validaciones_troncal = catalog.load("validaciones_troncal")

2022-03-29 17:21:21,818 - kedro.io.data_catalog - INFO - Loading data from `validaciones_troncal` (SparkDataSetMHD)...


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/29 17:22:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<IPython.core.display.Javascript object>

In [4]:
validaciones_troncal_pd = validaciones_troncal.toPandas()

<IPython.core.display.Javascript object>

In [7]:
validaciones_troncal_pd["Fecha_Transaccion"][0]

Timestamp('2020-01-01 10:33:28')

<IPython.core.display.Javascript object>